<a href="https://colab.research.google.com/github/MichaelPablo13/soccer_analysis_distributed_databases/blob/main/soccer_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count

# Start Spark
spark = SparkSession.builder \
    .appName("QuickTest") \
    .master("local[*]") \
    .getOrCreate()

# Create a simple DataFrame
data = [("Michael", 1), ("Copilot", 2)]
df = spark.createDataFrame(data, ["name", "id"])

# Show the DataFrame
df.show()

+-------+---+
|   name| id|
+-------+---+
|Michael|  1|
|Copilot|  2|
+-------+---+



In [18]:
csv_path = "/content/results(3).csv"
df = spark.read.csv(csv_path, header=True, inferSchema=True)
df.show()
df.describe().show()

+----------+----------------+-------------+--------------+--------------+--------------+---------+-------------------+-----------+
|      date|   home_teamName|away_teamName|home_scoreHome|away_scoreAway|tournamentName| cityCity|     countryCountry|neutralTRUE|
+----------+----------------+-------------+--------------+--------------+--------------+---------+-------------------+-----------+
|1872-11-30|        Scotland|      England|             0|             0|      Friendly|  Glasgow|           Scotland|      false|
|1873-03-08|         England|     Scotland|             4|             2|      Friendly|   London|            England|      false|
|1874-03-07|        Scotland|      England|             2|             1|      Friendly|  Glasgow|           Scotland|      false|
|1875-03-06|         England|     Scotland|             2|             2|      Friendly|   London|            England|      false|
|1876-03-04|        Scotland|      England|             3|             0|      Frie

In [19]:
# Total Matches Played
total_matches = df.count()
print(f"Total Matches Played: {total_matches}")

Total Matches Played: 40839


In [26]:
# Total Goals Scored by Each Team

home_goals = df.groupBy("home_teamName").sum("home_scoreHome").withColumnRenamed("sum(home_scoreHome)", "total_home_goals")
away_goals = df.groupBy("away_teamName").sum("away_scoreAway").withColumnRenamed("sum(away_scoreAway)", "total_away_goals")

total_goals = home_goals.join(away_goals, home_goals["home_teamName"] == away_goals["away_teamName"], "outer") \
  .fillna(0) \
  .withColumn("total_goals", col("total_home_goals") + col("total_away_goals")) \
  .select("home_teamName", "total_goals").orderBy(col("total_goals").desc())

total_goals.show()

+-------------+-----------+
|home_teamName|total_goals|
+-------------+-----------+
|      England|       2169|
|       Brazil|       2144|
|      Germany|       2104|
|       Sweden|       1998|
|      Hungary|       1882|
|    Argentina|       1816|
|  Netherlands|       1600|
|       Mexico|       1496|
|  South Korea|       1496|
|       France|       1480|
|      Denmark|       1398|
|      Uruguay|       1382|
|        Spain|       1380|
|      Austria|       1370|
|       Poland|       1361|
|        Italy|       1349|
|     Scotland|       1333|
|      Belgium|       1327|
|       Norway|       1198|
|       Russia|       1181|
+-------------+-----------+
only showing top 20 rows



# Task
Analyze the dataset to answer the following questions: 1- How many records are in the database? 2- How many unique home teams are there? 3- How many times did the home teams win? 4- How many times did the away teams win? 5- How many matches resulted in a draw? 6- How many matches were played in each country? 7- Which country had the most matches? 8- Which match had the highest number of goals? 9- What was the biggest rout? 10- How many games occurred in Brazil?

## Total records

### Subtask:
Calculate and display the total number of records in the DataFrame.


**Reasoning**:
The subtask is to calculate and display the total number of records in the DataFrame. The `count()` method is suitable for this purpose.



In [27]:
# Calculate the total number of records
total_records = df.count()

# Print the total number of records
print(f"Total number of records: {total_records}")

Total number of records: 40839


## Unique home teams

### Subtask:
Find and display the count of unique home teams.


**Reasoning**:
Select the home_teamName column, get distinct values, count them, and print the result.



In [28]:
# Select the home_teamName column, get distinct values, and count them
unique_home_teams_count = df.select("home_teamName").distinct().count()

# Print the count of unique home teams
print(f"Number of unique home teams: {unique_home_teams_count}")

Number of unique home teams: 309


## Home team wins

### Subtask:
Calculate and display the number of matches where the home team won.


**Reasoning**:
Filter the DataFrame to find matches where the home team won and then count the results.



In [29]:
home_wins_df = df.filter(df.home_scoreHome > df.away_scoreAway)
home_wins_count = home_wins_df.count()
print(f"Number of home team wins: {home_wins_count}")

Number of home team wins: 19864


## Away team wins

### Subtask:
Calculate and display the number of matches where the away team won.


**Reasoning**:
Filter the DataFrame to find matches where the away team won and then count the results.



In [30]:
away_wins_df = df.filter(df.away_scoreAway > df.home_scoreHome)
away_wins_count = away_wins_df.count()
print(f"Number of away team wins: {away_wins_count}")

Number of away team wins: 11544


## Draws

### Subtask:
Calculate and display the number of matches that ended in a draw.


**Reasoning**:
Filter the DataFrame to find matches where the scores are equal and then count the resulting rows to get the number of draws.



In [31]:
# Filter for draws (home_scoreHome == away_scoreAway)
draws_df = df.filter(df.home_scoreHome == df.away_scoreAway)

# Count the number of draws
draws_count = draws_df.count()

# Print the number of draws
print(f"Number of draws: {draws_count}")

Number of draws: 9431


## Matches per country

### Subtask:
Group the data by country and count the number of matches in each country.


**Reasoning**:
Group the data by country and count the number of matches in each country, then display the first few rows.



In [35]:
from pyspark.sql.functions import col, when, count, desc

matches_per_country_df = df.groupBy("countryCountry").count().withColumnRenamed("count", "number_of_matches")
matches_per_country_df.orderBy(desc("number_of_matches")).show()

+--------------------+-----------------+
|      countryCountry|number_of_matches|
+--------------------+-----------------+
|       United States|             1144|
|              France|              801|
|             England|              687|
|            Malaysia|              644|
|              Sweden|              637|
|             Germany|              581|
|              Brazil|              529|
|               Spain|              517|
|            Thailand|              483|
|               Italy|              480|
|         Switzerland|              477|
|             Austria|              475|
|United Arab Emirates|              472|
|        South Africa|              470|
|               Qatar|              467|
|         South Korea|              453|
|           Argentina|              449|
|             Hungary|              431|
|               Chile|              405|
|             Belgium|              396|
+--------------------+-----------------+
only showing top

## Matches in Brazil

### Subtask:
Filter the dataset for matches played in Brazil and count them.

**Reasoning**:
Filter the DataFrame to include only matches where the country is Brazil and then count the resulting rows to find the number of games played in Brazil.

In [39]:
# Filter the DataFrame for matches played in Brazil
matches_in_brazil_df = df.filter(df.countryCountry == "Brazil")

# Count the number of matches played in Brazil
matches_in_brazil_count = matches_in_brazil_df.count()

# Print the number of matches played in Brazil
print(f"Number of matches played in Brazil: {matches_in_brazil_count}")

Number of matches played in Brazil: 529


## Largest Goal Difference

### Subtask:
Calculate and display the match with the largest goal difference.

**Reasoning**:
Calculate the absolute difference between home and away scores for each match, order the matches by this goal difference in descending order, and select the top match to find the largest rout.

In [38]:
from pyspark.sql.functions import abs as spark_abs

# Calculate the goal difference for each match
df_with_goal_difference = df.withColumn("goal_difference", spark_abs(col("home_scoreHome") - col("away_scoreAway")))

# Find the match with the largest goal difference
largest_rout = df_with_goal_difference.orderBy(desc("goal_difference")).first()

# Print the match with the largest goal difference
print("Largest Rout:")
print(f"Date: {largest_rout['date']}")
print(f"Home Team: {largest_rout['home_teamName']}")
print(f"Away Team: {largest_rout['away_teamName']}")
print(f"Home Score: {largest_rout['home_scoreHome']}")
print(f"Away Score: {largest_rout['away_scoreAway']}")
print(f"Goal Difference: {largest_rout['goal_difference']}")
print(f"Tournament: {largest_rout['tournamentName']}")
print(f"City: {largest_rout['cityCity']}")
print(f"Country: {largest_rout['countryCountry']}")

Largest Rout:
Date: 2001-04-11
Home Team: Australia
Away Team: American Samoa
Home Score: 31
Away Score: 0
Goal Difference: 31
Tournament: FIFA World Cup qualification
City: Coffs Harbour
Country: Australia


## Match with most goals

### Subtask:
Find and display the match with the highest total goals.

**Reasoning**:
Calculate the total goals for each match by summing the home and away scores, order the matches by the total goals in descending order, and select the top match to find the match with the most goals.

In [37]:
# Calculate the total goals for each match
df_with_total_goals = df.withColumn("total_goals", col("home_scoreHome") + col("away_scoreAway"))

# Find the match with the highest number of goals
match_with_most_goals = df_with_total_goals.orderBy(desc("total_goals")).first()

# Print the match with the most goals
print("Match with the most goals:")
print(f"Date: {match_with_most_goals['date']}")
print(f"Home Team: {match_with_most_goals['home_teamName']}")
print(f"Away Team: {match_with_most_goals['away_teamName']}")
print(f"Home Score: {match_with_most_goals['home_scoreHome']}")
print(f"Away Score: {match_with_most_goals['away_scoreAway']}")
print(f"Total Goals: {match_with_most_goals['total_goals']}")
print(f"Tournament: {match_with_most_goals['tournamentName']}")
print(f"City: {match_with_most_goals['cityCity']}")
print(f"Country: {match_with_most_goals['countryCountry']}")

Match with the most goals:
Date: 2001-04-11
Home Team: Australia
Away Team: American Samoa
Home Score: 31
Away Score: 0
Total Goals: 31
Tournament: FIFA World Cup qualification
City: Coffs Harbour
Country: Australia


In [36]:
# Find the country with the most matches by taking the first row of the ordered DataFrame
country_with_most_matches = matches_per_country_df.orderBy(desc("number_of_matches")).first()

# Print the country with the most matches
print(f"Country with the most matches: {country_with_most_matches['countryCountry']} ({country_with_most_matches['number_of_matches']} matches)")

Country with the most matches: United States (1144 matches)


## Country with most matches

### Subtask:
Find and display the country with the highest number of matches.


**Reasoning**:
Ordering the DataFrame by the number of matches in descending order and selecting the top row will give the country with the highest number of matches, which directly addresses the subtask.



# Task
Analyze the provided dataset to determine the team with the best goal difference.

## Calculate home goal difference

### Subtask:
Calculate the goal difference for each team when playing at home.


**Reasoning**:
Calculate the goal difference for each team when playing at home by grouping by home team name, summing home scores and away scores, and subtracting the sums.



In [40]:
from pyspark.sql.functions import sum

# Calculate the goal difference for each team when playing at home
home_goal_difference_df = df.groupBy("home_teamName") \
    .agg(sum("home_scoreHome").alias("total_home_scores"),
         sum("away_scoreAway").alias("total_away_scores")) \
    .withColumn("home_goal_difference", col("total_home_scores") - col("total_away_scores")) \
    .select("home_teamName", "home_goal_difference")

# Display the result
home_goal_difference_df.show()

+-------------------+--------------------+
|      home_teamName|home_goal_difference|
+-------------------+--------------------+
|            Kabylia|                  -2|
|             Kernow|                   1|
|               Chad|                 -10|
|           Provence|                 -17|
|             Russia|                 350|
|           Paraguay|                 131|
|           Anguilla|                 -28|
|              Yemen|                 -23|
|U.S. Virgin Islands|                 -57|
|      South Ossetia|                  -9|
|     Chinese Taipei|                  11|
|            Senegal|                 198|
|             Madrid|                  -1|
|             Sweden|                 575|
|              Frøya|                  -6|
|           Kiribati|                 -63|
|           Ynys Môn|                  33|
|             Guyana|                  -4|
|        Philippines|                  -8|
|              Burma|                 187|
+----------

## Calculate away goal difference

### Subtask:
Calculate the goal difference for each team when playing away.


**Reasoning**:
Group the DataFrame by away team name, sum the away and home scores, calculate the away goal difference, select the required columns, and store the result in a new DataFrame.



In [41]:
from pyspark.sql.functions import sum

# Calculate the goal difference for each team when playing away
away_goal_difference_df = df.groupBy("away_teamName") \
    .agg(sum("away_scoreAway").alias("total_away_scores"),
         sum("home_scoreHome").alias("total_home_scores")) \
    .withColumn("away_goal_difference", col("total_away_scores") - col("total_home_scores")) \
    .select("away_teamName", "away_goal_difference")

# Display the result (optional, for verification)
away_goal_difference_df.show()

+-------------------+--------------------+
|      away_teamName|away_goal_difference|
+-------------------+--------------------+
|            Kabylia|                  -5|
|           Provence|                   4|
|               Chad|                 -76|
|             Kernow|                  11|
|             Russia|                 168|
|           Paraguay|                -223|
|           Anguilla|                -141|
|              Yemen|                -261|
|U.S. Virgin Islands|                -132|
|      South Ossetia|                   9|
|     Chinese Taipei|                -198|
|            Senegal|                 -22|
|             Sweden|                 120|
|              Frøya|                 -58|
|           Kiribati|                 -56|
|       Two Sicilies|                  -6|
|           Ynys Môn|                 -10|
|             Guyana|                -133|
|        Philippines|                -223|
|           Asturias|                   2|
+----------

## Combine goal differences

### Subtask:
Combine the home and away goal differences for each team.


**Reasoning**:
Combine the home and away goal difference dataframes using an outer join on the team names and fill the null values with 0. Then, display the result.



In [42]:
# Perform an outer join on the home and away goal difference DataFrames
combined_goal_difference_df = home_goal_difference_df.join(
    away_goal_difference_df,
    home_goal_difference_df["home_teamName"] == away_goal_difference_df["away_teamName"],
    "outer"
)

# Fill null values with 0 and select distinct team names
combined_goal_difference_df = combined_goal_difference_df \
    .select(
        col("home_teamName").alias("teamName"),
        col("home_goal_difference"),
        col("away_goal_difference")
    ) \
    .fillna(0)

# Display the resulting DataFrame
combined_goal_difference_df.show()

+-------------------+--------------------+--------------------+
|           teamName|home_goal_difference|away_goal_difference|
+-------------------+--------------------+--------------------+
|           Abkhazia|                  25|                  -1|
|        Afghanistan|                 -10|                 -90|
|            Albania|                  -7|                -212|
|           Alderney|                 -22|                 -38|
|            Algeria|                 261|                 -83|
|     American Samoa|                -116|                -154|
|          Andalusia|                  10|                   1|
|            Andorra|                -150|                -224|
|             Angola|                  98|                 -70|
|           Anguilla|                 -28|                -141|
|Antigua and Barbuda|                  25|                 -84|
|   Arameans Suryoye|                   2|                   2|
|          Argentina|                 72

## Calculate total goal difference

### Subtask:
Calculate the total goal difference for each team.


**Reasoning**:
Add a new column named `total_goal_difference` to the `combined_goal_difference_df` DataFrame by summing the values in the `home_goal_difference` and `away_goal_difference` columns for each row, and then display the updated DataFrame.



In [46]:
# Add a new column named total_goal_difference
combined_goal_difference_df = combined_goal_difference_df.withColumn(
    "total_goal_difference",
    col("home_goal_difference") + col("away_goal_difference")
)

# Display the updated DataFrame
combined_goal_difference_df.show()

+-------------------+--------------------+--------------------+---------------------+
|           teamName|home_goal_difference|away_goal_difference|total_goal_difference|
+-------------------+--------------------+--------------------+---------------------+
|           Abkhazia|                  25|                  -1|                   24|
|        Afghanistan|                 -10|                 -90|                 -100|
|            Albania|                  -7|                -212|                 -219|
|           Alderney|                 -22|                 -38|                  -60|
|            Algeria|                 261|                 -83|                  178|
|     American Samoa|                -116|                -154|                 -270|
|          Andalusia|                  10|                   1|                   11|
|            Andorra|                -150|                -224|                 -374|
|             Angola|                  98|            

## Find team with best goal difference

### Subtask:
Identify and display the team with the highest total goal difference.


**Reasoning**:
Ordering the DataFrame by the total goal difference in descending order and selecting the top row will give the team with the highest total goal difference, which directly addresses the subtask.



In [44]:
# Order the DataFrame by total_goal_difference in descending order and select the top row
team_with_highest_goal_difference = combined_goal_difference_df.orderBy(desc("total_goal_difference")).first()

# Print the team name and its total goal difference
print(f"Team with the highest total goal difference: {team_with_highest_goal_difference['teamName']} ({team_with_highest_goal_difference['total_goal_difference']})")

Team with the highest total goal difference: Brazil (1256)


## Summary:

### Data Analysis Key Findings

*   The team with the highest total goal difference is Brazil with a total goal difference of 1256.

### Insights or Next Steps

*   Investigate the distribution of goal differences across all teams to understand the overall competitive landscape.
*   Analyze the home and away goal differences separately for top teams to understand if their performance is skewed towards playing at home or away.
